In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf
import time
import numpy as np
import os.path
from transformer import CustomSchedule, Transformer, create_masks
# paths
checkpoint_path = "./checkpoints/train"
output_path = "./output"
data_path = './data'

In [0]:
# training parameters
BUFFER_SIZE = 20000
BATCH_SIZE = 64
MAX_LENGTH = 40 # use only training examples shorter than this
EPOCHS = 15
TRAIN_ON = 100 # percentage of data to train on
DICT_SIZE = 2**13 # this is likely too small
# model hyperparameters
num_layers = 4 # base transformer uses 6
d_model = 128 # base transformer uses 512
dff = 512 # base transformer uses 2048
num_heads = 8 # base transformer uses 8
dropout_rate = 0.1


In [0]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [0]:
examples, metadata = tfds.load('wmt14_translate/de-en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

In [20]:
if os.path.isfile(os.path.join(output_path, "tokenizer_en_" + str(DICT_SIZE) + ".subwords")):
    tokenizer_en = tfds.features.text.SubwordTextEncoder.load_from_file(os.path.join(output_path, "tokenizer_en_" + str(DICT_SIZE)))
else:
    tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
        (en.numpy() for de, en in train_examples), target_vocab_size=DICT_SIZE)
    tokenizer_en.save_to_file(os.path.join(output_path, "tokenizer_en_" + str(DICT_SIZE)))
if os.path.isfile(os.path.join(output_path, "tokenizer_de_" + str(DICT_SIZE) + ".subwords")):
    tokenizer_de = tfds.features.text.SubwordTextEncoder.load_from_file(os.path.join(output_path, "tokenizer_de_" + str(DICT_SIZE)))
else:
    tokenizer_de = tfds.features.text.SubwordTextEncoder.build_from_corpus(
        (de.numpy() for de, en in train_examples), target_vocab_size=DICT_SIZE)
    tokenizer_de.save_to_file(os.path.join(output_path, "tokenizer_de_" + str(DICT_SIZE)))

In [37]:
input_vocab_size = tokenizer_de.vocab_size + 2
target_vocab_size = tokenizer_en.vocab_size + 2

In [38]:
def filter_max_length(x, y, max_length=MAX_LENGTH):
  return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

def encode(lang1, lang2):
  lang1 = [tokenizer_de.vocab_size] + tokenizer_de.encode(
      lang1.numpy()) + [tokenizer_de.vocab_size+1]

  lang2 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang2.numpy()) + [tokenizer_en.vocab_size+1]
  
  return lang1, lang2
  
def tf_encode(de, en):
  return tf.py_function(encode, [de, en], [tf.int64, tf.int64])

In [39]:
split = tfds.Split.TRAIN.subsplit(tfds.percent[:TRAIN_ON])

examples, metadata = tfds.load('wmt14_translate/de-en', data_dir=data_path, with_info=True,
                               as_supervised=True, split=[split, 'validation'])
train_examples, val_examples = examples[0], examples[1]

train_dataset = train_examples.map(tf_encode)
train_dataset = train_dataset.filter(filter_max_length)
# cache the dataset to memory to get a speedup while reading from it.
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(
    BATCH_SIZE, padded_shapes=([-1], [-1]))
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


val_dataset = val_examples.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(
    BATCH_SIZE, padded_shapes=([-1], [-1]))

ExtractError: Error while extracting ./data\downloads\statmt.org_wmt13_traini-parall-europa-v7AiTHxxDIoGPf2JOwzAgwIC1h9MdcF-uOMYNhA9J9luc.tgz to ./data\downloads\extracted\TAR_GZ.statmt.org_wmt13_traini-parall-europa-v7AiTHxxDIoGPf2JOwzAgwIC1h9MdcF-uOMYNhA9J9luc.tgz : Failed to create a directory: ./data\downloads\extracted\TAR_GZ.statmt.org_wmt13_traini-parall-europa-v7AiTHxxDIoGPf2JOwzAgwIC1h9MdcF-uOMYNhA9J9luc.tgz.incomplete_a8f39f2bc0964bf5bd164251b4764de5\training; No such file or directory

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf
import time
import numpy as np
import os.path
from transformer import CustomSchedule, Transformer, create_masks

W1214 14:00:20.518681 17164 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

